## CPU-bound в async

In [4]:
import asyncio

In [5]:
async def foo():
    print('🚀 Running foo...')
    for i in range(7_000):          ## CPU-bound task блокирует поток!
        i**i
        if i % 100 == 0:            # Раз в 100 итераций
            await asyncio.sleep(0)  # Переключаем контекст
    print('✅ foo: OK')
    return 'foo'

async def bar():
    print('🚀 Running bar...')
    await asyncio.sleep(1)
    print('✅ bar: OK')
    return 'bar'

res = await asyncio.gather(foo(), bar())

print(res)

🚀 Running foo...
🚀 Running bar...
✅ bar: OK
✅ foo: OK
['foo', 'bar']


## Lock
Приостанавливает и возобновляет корутину

In [2]:
from asyncio import Lock

In [18]:
async def access(lock: Lock, number):
    
    await lock.acquire()
    print(f'🔒 Locked: #{number}')

    try:
        await asyncio.sleep(2)
    finally:
        print(f'🔓 Unlocked: #{number}\n')
        lock.release()
    
    return number



lock = asyncio.Lock()

tasks = [asyncio.create_task(access(lock, i)) for i in range(2)] + [asyncio.create_task(bar())]

await asyncio.gather(*tasks)

🔒 Locked: #0
🚀 Running bar...
✅ bar: OK
🔓 Unlocked: #0

🔒 Locked: #1
🔓 Unlocked: #1



[0, 1, 'bar']

### Предотвращение "состояния гонки"

In [32]:
# Обеспечение атомарности операций
shared_counter = 0
lock = asyncio.Lock()

async def inc_counter(s: int):
    global shared_counter
    async with lock:
        print(f'🔒 Locked: #{s}')

        current_counter_value = shared_counter
        current_counter_value += 1

        await asyncio.sleep(s)
        shared_counter = current_counter_value
    print(f'🔓 Unlocked: #{s}\n')

await asyncio.gather(*[inc_counter(i) for i in range(1,3)])

shared_counter

🔒 Locked: #1
🔓 Unlocked: #1

🔒 Locked: #2
🔓 Unlocked: #2



2